In [1]:
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import pandas as pd 
from data import L_D_Hours, Wind_Hours, PC_DA, P_max, Load_Demand, Cost_Generation, Wind_Farm_Power, Wind, T, N, P_min, C_up, C_down, RE_up, RE_down, C_plus, C_min # Import data from data.py


h = 7 
UP_req = 0.15  # 15% upward reserve
DOWN_req = 0.10  # 10% downward reserve

# Defining the number of demand nodes and wind farms 
Dem = len(L_D_Hours)  # Number of demand nodes
Wind = len(Wind_Hours)  # Number of wind farms
Gen = len(P_max)  # Number of generators

# Load and wind profile for hour h
Load = np.zeros(Dem)
Load[:] = L_D_Hours[:, h]     # load demand at hour h
Wind_profile = np.zeros(Wind)
Wind_profile[:] = Wind_Hours[:, h]  # Wind production at hour h

# Load bid data from a CSV file
file_path = "day_ahead_bids.csv"
df = pd.read_csv(file_path)

# Ensure correct column names
df.columns = ["Hour", "Bid Price", "Bid Amount"]

# Convert bid data into a dictionary grouped by hour
bid_data = {}
for hour, group in df.groupby("Hour"):
    bid_data[hour] = group[["Bid Price", "Bid Amount"]].to_dict(orient="records")


bid_prices = {} 

for hour in range(1, T + 1):
    # Get the sorted bids by price for the current hour
    sorted_bids = sorted(bid_data.get(hour, []), key=lambda x: x["Bid Price"])
    
    # Allocate bid prices to demand nodes based on sorted bids
    total_bid_amount = 0
    bid_prices[hour] = []
    
    for bid in sorted_bids:
        bid_price = bid["Bid Price"]
        bid_amount = bid["Bid Amount"]
        
        # Allocate the bid price to demand nodes 
        for _ in range(int(bid_amount)):  
            bid_prices[hour].append(bid_price)
    
    if len(bid_prices[hour]) < Dem:
        # Fill the remaining demand nodes with the highest available price
        highest_price = sorted_bids[-1]["Bid Price"]
        bid_prices[hour].extend([highest_price] * (Dem - len(bid_prices[hour])))

# Step 1: Reserve Market Model

# Create the reserve model
reserve_model = ConcreteModel()

# Decision variables for the reserve market
reserve_model.R_Gen_UP = Var(range(Gen), domain=NonNegativeReals)  # Up reserve of conventional generators
reserve_model.R_Gen_Down = Var(range(Gen), domain=NonNegativeReals)  # Down reserve of conventional generators

# Objective function: Minimize the reserve procurement cost
def reserve_objective_rule(model):
    return sum(C_plus[gen] * model.R_Gen_UP[gen] for gen in range(Gen)) + \
           sum(C_min[gen] * model.R_Gen_Down[gen] for gen in range(Gen))

reserve_model.obj = Objective(rule=reserve_objective_rule, sense=minimize)

# Constraints

# Maximum reserve constraint for up and down reserves
def max_up_reserve_rule(model, gen):
    return model.R_Gen_UP[gen] <= RE_up[gen]
reserve_model.max_up_reserve = Constraint(range(Gen), rule=max_up_reserve_rule)

def max_down_reserve_rule(model, gen):
    return model.R_Gen_Down[gen] <= RE_down[gen]
reserve_model.max_down_reserve = Constraint(range(Gen), rule=max_down_reserve_rule)

# Total reserve capacity (up and down reserves cannot exceed total capacity)
def total_reserve_capacity_rule(model):
    return sum(model.R_Gen_UP[gen] + model.R_Gen_Down[gen] for gen in range(Gen)) <= sum(P_max[gen] for gen in range(Gen))
reserve_model.total_reserve_capacity = Constraint(rule=total_reserve_capacity_rule)

# TSO reserve requirement (up and down reserve)
def reserve_up_rule(model):
    total_demand = sum(L_D_Hours[dem, h-1] for dem in range(Dem)) 
    return sum(model.R_Gen_UP[gen] for gen in range(Gen)) == UP_req * total_demand
reserve_model.reserve_up = Constraint(rule=reserve_up_rule)

def reserve_down_rule(model):
    total_demand = sum(L_D_Hours[dem, h-1] for dem in range(Dem))  
    return sum(model.R_Gen_Down[gen] for gen in range(Gen)) == DOWN_req * total_demand
reserve_model.reserve_down = Constraint(rule=reserve_down_rule)

# Solve the reserve model
solver = SolverFactory('glpk')
solver.solve(reserve_model)

# Step 2: Day-Ahead Market Model for hour h, using the reserve market results

# Use reserve market results for the day-ahead model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Decision Variables
model.P_g = Var(range(Gen), domain=NonNegativeReals)  # Production of conventional generators
model.P_w = Var(range(Wind), domain=NonNegativeReals)  # Production of wind farms
model.P_d = Var(range(Dem), domain=NonNegativeReals)  # Demand

# Objective function: Social welfare with the reserve costs considered
def objective_rule(model):
    total_revenue = sum(bid_prices[h][dem] * model.P_d[dem] for dem in range(Dem))
    total_cost = sum(PC_DA[gen] * model.P_g[gen] for gen in range(Gen)) + \
                 sum(0 * model.P_w[wind] for wind in range(Wind))  # No cost for wind
    total_reserve_cost = sum(C_up[gen] * reserve_model.R_Gen_UP[gen].value for gen in range(Gen)) + \
                         sum(C_down[gen] * reserve_model.R_Gen_Down[gen].value for gen in range(Gen))
    return total_revenue - total_cost - total_reserve_cost

model.obj = Objective(rule=objective_rule, sense=maximize)

# Constraints

# Capacity constraints for conventional generation
def conventional_generation_rule(model, gen):
    return model.P_g[gen] <= P_max[gen] - reserve_model.R_Gen_UP[gen].value
model.conventional_generation = Constraint(range(Gen), rule=conventional_generation_rule)

def conventional_generation_rule1(model, gen):
    return reserve_model.R_Gen_Down[gen].value <= model.P_g[gen]
model.conventional_generation1 = Constraint(range(Gen), rule=conventional_generation_rule1)

# Capacity constraints for wind generation
def wind_generation_rule(model, wind):
    return model.P_w[wind] <= Wind_profile[wind]
model.wind_generation = Constraint(range(Wind), rule=wind_generation_rule)

# Demand constraint
def demand_rule(model, dem):
    return model.P_d[dem] <= L_D_Hours[dem, h-1]  
model.demand = Constraint(range(Dem), rule=demand_rule)

# Balance constraint (total demand = total generation)
def balance_rule(model):
    return sum(model.P_d[dem] for dem in range(Dem)) == \
           sum(model.P_g[gen] for gen in range(Gen)) + \
           sum(model.P_w[wind] for wind in range(Wind))
model.balance = Constraint(rule=balance_rule)

# Solve the day-ahead model
solver.solve(model)

# Results

# Retrieve optimal objective value (social welfare)
print(f"Optimal objective value (Social Welfare): {model.obj()}")

# Display the production of wind farms and conventional generators
for wind in range(Wind):
    print(f"Wind Farm {wind} production: {model.P_w[wind].value} MW")

for gen in range(Gen):
    print(f"Conventional Generator {gen} production: {model.P_g[gen].value} MW")

# Market Clearing Price 
MCP = model.dual[model.balance]
print(f"Market Clearing Price (MCP): {MCP} $/MWh")

# Calculate the profit for each generator
for gen in range(Gen):
    revenue = MCP * model.P_g[gen].value
    cost = PC_DA[gen] * model.P_g[gen].value
    profit = revenue - cost
    print(f"Profit for Conventional Generator {gen}: ${profit}")

# Step 4: Reserve Market Cost
print("\nReserve Market Costs:")

# Reserve cost for upward reserve
reserve_up_cost = sum(C_up[gen] * reserve_model.R_Gen_UP[gen].value for gen in range(Gen))
# Reserve cost for downward reserve
reserve_down_cost = sum(C_down[gen] * reserve_model.R_Gen_Down[gen].value for gen in range(Gen))

# Total reserve cost
total_reserve_cost = reserve_up_cost + reserve_down_cost
print(f"Upward Reserve Market Cost: ${reserve_up_cost}")
print(f"Downward Reserve Market Cost: ${reserve_down_cost}")
print(f"Total Reserve Market Cost: ${total_reserve_cost}")

# Printing up and down reserve values for each generator
print("\nUp and Down Reserve of the Generators:")

for gen in range(Gen):
    up_reserve = reserve_model.R_Gen_UP[gen].value
    down_reserve = reserve_model.R_Gen_Down[gen].value
    print(f"Generator {gen}: Up reserve = {up_reserve} MW, Down reserve = {down_reserve} MW")

Optimal objective value (Social Welfare): 64203.753729488
Wind Farm 0 production: 143.1758086 MW
Wind Farm 1 production: 146.418634 MW
Wind Farm 2 production: 87.212025 MW
Wind Farm 3 production: 121.150513 MW
Wind Farm 4 production: 127.118818 MW
Wind Farm 5 production: 103.6143558 MW
Conventional Generator 0 production: 0.0 MW
Conventional Generator 1 production: 36.13 MW
Conventional Generator 2 production: 0.0 MW
Conventional Generator 3 production: 0.0 MW
Conventional Generator 4 production: 0.0 MW
Conventional Generator 5 production: 30.0 MW
Conventional Generator 6 production: 30.0 MW
Conventional Generator 7 production: 336.4798456 MW
Conventional Generator 8 production: 400.0 MW
Conventional Generator 9 production: 300.0 MW
Conventional Generator 10 production: 60.0 MW
Conventional Generator 11 production: 40.0 MW
Market Clearing Price (MCP): 6.02 $/MWh
Profit for Conventional Generator 0: $0.0
Profit for Conventional Generator 1: $-263.749
Profit for Conventional Generator 2: